# Trace API

<div class='subtitle'>Learn to traverse agent traces to write precise assertions</div>

An agent run results in a trace of events and actions that correspond to the actions and responses of the agent. For effective testing, we need to inspect the trace to ensure we are checking our test assertions against the correct parts of the trace.

For this, `testing` provides the `Trace` data structure to inspect a given trace:


In [ ]:
from invariant_runner.custom_types.trace import Trace

trace = Trace(trace=[
    {"role": "user", "content": "Hello there"},
    {"role": "assistant", "content": "Hello there", "tool_calls": [
        {
            "type": "function",
            "function": {
                "name": "greet",
                "arguments": {
                    "name": "there"
                }
            }
        }
    ]},
    {"role": "user", "content": "I need help with something."},
])

## Selecting Messages

A `Trace` object can be used to select specific messages from the trace. This is useful for selecting messages that are relevant to the test assertions.

In [4]:
# select the first trace message
trace.messages(0)

InvariantDict{'role': 'user', 'content': 'Hello there'} at 0

In [5]:
# select all user messages
trace.messages(role="user")

InvariantList[
  {'role': 'user', 'content': 'Hello there'}
  {'role': 'user', 'content': 'I need help with something.'}
] at [['0'], ['2']]

In [7]:
# select the message with 'something' in the content
trace.messages(content=lambda c: 'something' in c)

InvariantList[
  {'role': 'user', 'content': 'I need help with something.'}
] at [['2']]

**Assertion Localization**: On the one hand, the `trace.messages(...)` selector function gives you a convenient way to select messages from the trace. In addition to this, however, it will also always keep track of the exact path of the resulting objects in the trace.

This is useful for debugging and to localize assertion failures, down to the exact agent event that is causing the failure.

This also applies to multiple levels of nested structures, e.g. when selecting the content of a specific message only:

In [8]:
# selecting content from the 2nd message in the trace
trace.messages(2)["content"]

InvariantString(value=I need help with something., addresses=['2.content:0-27'])

## Selecting Tool Calls

Similar to selecting messages, you can also select just tool calls from the trace.

In [9]:
greet_calls = trace.tool_calls(name="greet")
print(greet_calls[0])

InvariantDict{'type': 'function', 'function': {'name': 'greet', 'arguments': {'name': 'there'}}} at ['1.tool_calls.0']


Again, all accesses are tracked and include the exact source path and range in the trace (e.g. `1.tool_calls.0` here).

> Note that even though you can select `.tool_calls()` directly on `name` and `arguments`, the returned object is always of `{'type': 'function', 'function': { ... }}` shape.

## Accessors: Deriving Extra Information

After selecting individual messages or tool calls, you can also derive extra information from them. This is useful for computing similarity metrics, comparisons or other derived information.

For instance, to compute the length of some messages's `content`, the following method can be used:

In [9]:
# check the length of the resposne
trace.messages(0)["content"].len()

InvariantNumber(value=11, addresses=['0.content:0-11'])

As we compute extra information, like the length of a string, the path in the trace is still tracked and included in the result. This is useful for debugging and to localize assertion failures, down to the exact agent event that is causing the failure.

### Similarity Metrics

In [16]:
# check that the first message is not too far from "Hello there"
trace.messages(0)["content"].levenshtein("Hello there")

InvariantNumber(value=1.0, addresses=['0.content:0-11'])

In [5]:
# check that the first message is not too far from "Hello there" in embeddings
trace.messages(0)["content"].is_similar("Greetings")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


InvariantBool(value=True, addresses=['0.content:0-11'])

InvariantNumber(value=11, addresses=['0.content:0-11'])

Here, member functions are used to access to compute derived values such as Levenshtein distance and tokenization of the text. 

These so-called *transformer* functions on trace objects have the special property that they are fully tracked by the Invariant Testing library: Any result of a transformer function can be tracked back fully to exact input trace objects that were used to compute it.

This is especially helpful to track assertion failures back to the exact source of the problem.

For instance, when comparing `trace.messages(1001)["content"].len() < 10`, the assertion failure can be attributed to the exact (1001th) message content that caused the failure, rather than just failing on the entire trace as a whole.